### Star Ratings Results
- Input from AimBig notebook, stat_regression 
- 6 steps of the star rating results
- GroupBy object = (Contract ID, ESA Code, Speciality_site_type_code)

In [40]:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# reading in file with numerator/denominator for each participant, performance measure against contract
df_merged = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_final.csv')
# creating groupBy object 
df_merged = df_merged.drop_duplicates()
#df_group = df_merged.groupby(['CONTRACT_ID', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'])

df_merged.columns

df_merged = df_merged.drop(columns=['Unnamed: 0'])

Columns missing due to no outcomes = 'Num_26_path', 'Num_26_wrkast', 'Den_26_path', 'Den_26_wrkast', 'Den_52_path', 'Num_52_path'
- set as 0 

In [41]:
df_merged

,JOB_SEEKER_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,9386301003,0,0,AimBig Employment WOY WOY,0212812K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8653691003,0,0,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
2,7870290004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
3,6638331004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2580218019,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,1922050119,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
1449,7877870004,0,0,AimBig Employment DALBY,0212897K,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
1450,3825170009,0,4,AimBig Employment FRANKSTON,0212818F,MENH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1451,4123062109,0,0,AimBig Employment ROCKHAMPTON CITY,0212801H,MUSK,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0


In [42]:
print(df_merged.isna().sum())

JOB_SEEKER_ID                0
Num_13                       0
Den_13                       0
Site_Name                    0
Contract_ID                  0
specialist_site_type_code    0
Num_26_full                  0
Den_26_full                  0
Num_26_path                  0
Den_26_path                  0
Num_26_wrkast                0
Den_26_wrkast                0
Num_52_full                  0
Den_52_full                  0
Num_52_path                  0
Den_52_path                  0
dtype: int64


### Step 1) Calculating Actual Performance
#### Actual Performance
groupBy key = (Contract ID, ESA Code, Speciality_site_type_code)

##### Pre-Quarter

- Count NaNs -> place as 0

In [43]:
df_merged = df_merged.fillna(value = 0)
df_merged

,JOB_SEEKER_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,9386301003,0,0,AimBig Employment WOY WOY,0212812K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8653691003,0,0,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
2,7870290004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
3,6638331004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2580218019,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,1922050119,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
1449,7877870004,0,0,AimBig Employment DALBY,0212897K,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
1450,3825170009,0,4,AimBig Employment FRANKSTON,0212818F,MENH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1451,4123062109,0,0,AimBig Employment ROCKHAMPTON CITY,0212801H,MUSK,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0


In [44]:
df_merged.columns

Index(['JOB_SEEKER_ID', 'Num_13', 'Den_13', 'Site_Name', 'Contract_ID',
       'specialist_site_type_code', 'Num_26_full', 'Den_26_full',
       'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'],
      dtype='object')

In [45]:
df_merged = df_merged.rename(columns={'Num_13' : 'Num_13_full', 'Den_13' : 'Den_13_full'})

### Aggregating to Site Level 

In [46]:
df_group = df_merged.groupby(['Contract_ID', 'Site_Name',  'specialist_site_type_code'])['Num_13_full', 'Den_13_full', 'Site_Name', 'Contract_ID',
       'specialist_site_type_code', 'Num_26_full', 'Den_26_full',
       'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()

df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100




#### Renaming df_group -> df_pre_actual

In [47]:
df_pre_actual = df_group
df_pre_actual.to_csv('C:/Users/AManalo/star_ratings_new/Output Files/test_sites.csv')

In [48]:
def get_actPerf(type, df_group):
    '''Input:
        - type: 'pre' or 'post' quarter
        - df_group: groupBy object that contains site participants with num/den
        Output:
        - returns dataframe with pre/post quarter actual performance for each Contract'''
    
    if type == 'pre':
        df_group['preAct13_full'] = 0 # initialise pre_actual_rate column
        df_group['preAct26_path'] = 0
        df_group['preAct26_full'] = 0
        df_group['preAct26_wrkast'] = 0
        df_group['preAct52_full'] = 0
        df_group['preAct52_path'] = 0
        

    elif type == 'post':
        df_group['postAct13_full'] = 0 # initialise post_actual_rate column
        df_group['postAct26_path'] = 0
        df_group['postAct26_full'] = 0
        df_group['postAct26_wrkast'] = 0
        df_group['postAct52_full'] = 0
        df_group['postAct52_path'] = 0

    

    return df_group

##### Post-Quarter

In [49]:
# reading in the merged file
df_merge_post = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_post.csv')
df_merge_post = df_merge_post.fillna(value = 0)
df_merge_post = df_merge_post.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_CODE' : 'Site_Code', 'SITE_DESCRIPTION': 'Site_Name', 'SPECIALIST_SITE_TYPE_CODE' : 'specialist_site_type_code'})
df_merge_post.head(5)

,Unnamed: 0,Program,JOB_SEEKER_ID,Contract_ID,Site_Code,Site_Name,ESA Code,ESA Name,State,specialist_site_type_code,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2,2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


This is WITHOUT 52 week outcomes -> ONLY FOR TESTING 13 and 26 week

In [50]:
df_merge_post = df_merge_post.rename(columns={'Num_13' : 'Num_13_full', 'Den_13' : 'Den_13_full'})

In [51]:
df_merge_post

,Unnamed: 0,Program,JOB_SEEKER_ID,Contract_ID,Site_Code,Site_Name,ESA Code,ESA Name,State,specialist_site_type_code,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2,2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5307,5308,DMS,2730016019,0212841E,Y947,AimBig Employment MERRYLANDS,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5308,5309,DMS,2696040,0212857B,ML77,AimBig Employment WOLLONGONG,4WOL,South East Sydney,NSW,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5309,5310,DMS,5892574007,0212825E,MM77,AimBig Employment MELBOURNE,4YAR,Northern Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5310,5311,DMS,832617003,0212823C,YA97,AimBig Employment Pty Ltd ST ALBANS,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
df_group = df_merge_post.groupby(['Contract_ID', 'Site_Name', 'specialist_site_type_code', 'JOB_SEEKER_ID'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()
df_group = df_merge_post.groupby(['Contract_ID', 'Site_Name', 'specialist_site_type_code'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()




Calculating Actual Performance

In [53]:
# renaming df_13 ->



df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100

df_group = df_group.fillna(0.5)

df_group.to_csv('C:/Users/AManalo/star_ratings_new/df_group_test.csv')

In [54]:
df_group

Num_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               2.0   
            AimBig Employment PROSPECT   MENH                              12.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               3.0   
0212791K    AimBig Employment MILE END   MENH                               8.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               1.0   
0212904D    AimBig Employment HURSTVILLE AALL                              13.0   
            AimBig Employment MIRANDA    AALL                               6.0   
            AimBig Employment ROCKDALE   AALL                               4.0   

                                                                    Den_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               2.0   
            AimBig Employment PROSPECT   MENH                              18.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               4.0   
0212791K    AimBig Employment MILE END   MENH                              13.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               1.0   
0212904D    AimBig Employment HURSTVILLE AALL                              19.0   
            AimBig Employment MIRANDA    AALL                               9.0   
            AimBig Employment ROCKDALE   AALL                               7.0   

                                                                    Num_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               1.0   
            AimBig Employment PROSPECT   MENH                               8.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               1.0   
0212791K    AimBig Employment MILE END   MENH                               6.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               0.0   
0212904D    AimBig Employment HURSTVILLE AALL                               7.0   
            AimBig Employment MIRANDA    AALL                               3.0   
            AimBig Employment ROCKDALE   AALL                               0.0   

                                                                    Den_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               1.0   
            AimBig Employment PROSPECT   MENH                              13.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               1.0   
0212791K    AimBig Employment MILE END   MENH                               9.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               0.0  

## Step 2: Expected Performance

Using statistical regression, which takes account of variable Participant and labour market characteristics and the number of Participants in the denominator post-quarterisation, the Star Ratings model calculates the number of outcomes that the Contract could be expected to achieve.

In [55]:

df_group.columns

Index(['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full',
       'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path',
       'actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path'],
      dtype='object')

In [56]:
df_merge_post

,Unnamed: 0,Program,JOB_SEEKER_ID,Contract_ID,Site_Code,Site_Name,ESA Code,ESA Name,State,specialist_site_type_code,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2,2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5307,5308,DMS,2730016019,0212841E,Y947,AimBig Employment MERRYLANDS,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5308,5309,DMS,2696040,0212857B,ML77,AimBig Employment WOLLONGONG,4WOL,South East Sydney,NSW,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5309,5310,DMS,5892574007,0212825E,MM77,AimBig Employment MELBOURNE,4YAR,Northern Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5310,5311,DMS,832617003,0212823C,YA97,AimBig Employment Pty Ltd ST ALBANS,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# Get expected columns from df_merged
df_merge_post.reset_index()
df_merge_post.columns

df_exp = df_merge_post[['JOB_SEEKER_ID', 'Contract_ID',  'Site_Name', 'specialist_site_type_code', 'Num_13_full_exp', 'Num_26_full_exp', 'Num_26_path_exp', 'Num_26_wrkast_exp', 'Num_52_full_exp', 'Num_52_path_exp']]
       

Merging df_exp with the denominator from post quarterisation outcomes

In [58]:
df_merge_post.columns

Index(['Unnamed: 0', 'Program', 'JOB_SEEKER_ID', 'Contract_ID', 'Site_Code',
       'Site_Name', 'ESA Code', 'ESA Name', 'State',
       'specialist_site_type_code', 'Num_13_full', 'Den_13_full',
       'Num_13_full_exp', 'index', 'Num_26_full', 'Den_26_full',
       'Num_26_full_exp', 'Num_26_path', 'Den_26_path', 'Num_26_path_exp',
       'Num_26_wrkast', 'Den_26_wrkast', 'Num_26_wrkast_exp', 'Num_52_full',
       'Den_52_full', 'Num_52_full_exp', 'Num_52_path', 'Den_52_path',
       'Num_52_path_exp'],
      dtype='object')

In [59]:
df_exp.to_csv('C:/Users/AManalo/star_ratings_new/df_exp.csv')

In [60]:
df_merge_post
df_post_den = df_merge_post[["JOB_SEEKER_ID", "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path"]]
df_exp_per = df_exp.merge(df_post_den, on=['JOB_SEEKER_ID'], how='inner')


df_exp_per = df_exp_per.groupby(['Contract_ID', 'Site_Name',  'specialist_site_type_code'])[ "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path", 'Num_13_full_exp', 'Num_26_full_exp', 'Num_52_full_exp', 'Num_52_path_exp', 'Num_26_wrkast_exp', 'Num_26_path_exp'].sum()



Calculating expected performance rate
- for each performance measure

In [61]:

df_exp_per['exPer_13_full'] = df_exp_per['Den_13_full']/df_exp_per['Num_13_full_exp']
df_exp_per['exPer_26_full'] = df_exp_per['Den_26_full']/df_exp_per['Num_26_full_exp']
df_exp_per['exPer_26_path'] = df_exp_per['Den_26_path']/df_exp_per['Num_26_path_exp']
df_exp_per['exPer_26_wrkast'] = df_exp_per['Den_26_wrkast']/df_exp_per['Num_26_wrkast_exp']
df_exp_per['exPer_52_full'] = df_exp_per['Den_52_full']/df_exp_per['Num_52_full_exp']
df_exp_per['exPer_52_path'] = df_exp_per['Den_52_path']/df_exp_per['Num_52_path_exp']

df_exp_per['exPer_26_path'] = np.where(df_exp_per['exPer_26_path'] == 0, 0.5, df_exp_per['exPer_26_path'])
df_exp_per['exPer_52_path'] = np.where(df_exp_per['exPer_52_path'] == 0, 0.5, df_exp_per['exPer_52_path'])
df_exp_per['exPer_26_wrkast'] = np.where(df_exp_per['exPer_26_wrkast'] == 0, 0.5, df_exp_per['exPer_26_wrkast'])
df_exp_per = df_exp_per.replace([0, np.nan, np.inf, -np.inf], 0.5)

Read in the Sub368 file and add merge this with the df_exp_per
- The sub368 averages end in column '_avg'

In [62]:
df_sub = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_sub.csv')
df_sub = df_sub.groupby(['CONTRACT_ID', 'SITE_CODE', 'SPECIALTY_CD', 'SITE_NAME'])['52_full_avg', '52_path_avg',  '26_wrkast_avg', '26_full_avg', '26_path_avg', '13_full_avg'].sum()
df_sub = df_sub.reset_index()
df_sub = df_sub.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_NAME' : 'Site_Name', 'SPECIALTY_CD' : 'specialist_site_type_code'})

Merge the df_sub columns with the df_exp_per

In [63]:
df_exp_per = df_exp_per.reset_index()
df_exp_per = df_exp_per.merge(df_sub, on=['Site_Name', 'Contract_ID', 'specialist_site_type_code'], how='inner')


## Step 3: Calculate the Performance Measure Scores (Ratio of Actual Outcomes : Expected Outcomes)

Combined 13, 25, 52 week scores are calculated based on the weightings of each sub type measure. 
Actual rate: actual rate (post-quarter)
Expected rate: expected performance rate


To-Fix: Need to adjust the weights for each performance measure IF the denominator in the pre-quarter outcomes are 0 or very small. 

- ESS: min 20 participants (combined) in the pre-quarterisation denominators for the following: 13-week full outcomes (min 5 participants), ongoing support  
- DMS: min 20 participants in the pre-quarterisation denominator for 13 week full outcomes measure

In [64]:
df_group.reset_index()

,Contract_ID,Site_Name,specialist_site_type_code,Num_13_full,Den_13_full,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,...,Num_52_full,Den_52_full,Num_52_path,Den_52_path,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path
0,0212789F,AimBig Employment NORWOOD,MENH,2.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.000000,100.000000,0.5,0.5,0.500000,0.5
1,0212789F,AimBig Employment PROSPECT,MENH,12.0,18.0,8.0,13.0,0.0,5.0,0.0,...,3.0,9.0,0.0,0.0,66.666667,61.538462,0.0,0.5,33.333333,0.5
2,0212790K,AimBig Employment NORWOOD,MUSK,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.000000,100.000000,0.5,0.5,0.500000,0.5
3,0212790K,AimBig Employment PROSPECT,MUSK,3.0,4.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,75.000000,100.000000,0.5,0.5,100.000000,0.5
4,0212791K,AimBig Employment MILE END,MENH,8.0,13.0,6.0,9.0,1.0,4.0,0.0,...,1.0,4.0,0.0,0.0,61.538462,66.666667,25.0,0.5,25.000000,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0212903C,Aimbig Employment RICHMOND,AALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.500000,0.500000,0.5,0.5,0.500000,0.5
209,0212903C,Aimbig Employment WINDSOR,AALL,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.000000,0.500000,0.5,0.5,0.500000,0.5
210,0212904D,AimBig Employment HURSTVILLE,AALL,13.0,19.0,7.0,10.0,0.0,3.0,0.0,...,0.0,2.0,0.0,0.0,68.421053,70.000000,0.0,0.5,0.000000,0.5
211,0212904D,AimBig Employment MIRANDA,AALL,6.0,9.0,3.0,5.0,0.0,2.0,0.0,...,0.0,2.0,0.0,0.0,66.666667,60.000000,0.0,0.5,0.000000,0.5


In [65]:
df_exp_per.to_csv('C:/Users/AManalo/star_ratings_new/expected_outcomes.csv')

In [66]:
df_exp_per

,Contract_ID,Site_Name,specialist_site_type_code,Den_13_full,Den_26_full,Den_26_path,Den_26_wrkast,Den_52_full,Den_52_path,Num_13_full_exp,...,exPer_26_wrkast,exPer_52_full,exPer_52_path,SITE_CODE,52_full_avg,52_path_avg,26_wrkast_avg,26_full_avg,26_path_avg,13_full_avg
0,0212789F,AimBig Employment PROSPECT,MENH,18.0,13.0,5.0,0.5,9.0,0.5,6.0,...,0.5,0.900000,0.5,MO87,15.7,1.1,71.9,22.8,1.2,24.0
1,0212790K,AimBig Employment PROSPECT,MUSK,4.0,1.0,0.5,0.5,1.0,0.5,1.0,...,0.5,0.500000,0.5,MO87,15.7,1.1,71.9,22.8,1.2,24.0
2,0212791K,AimBig Employment MILE END,MENH,13.0,9.0,4.0,0.5,4.0,0.5,6.0,...,0.5,0.500000,0.5,MO77,15.7,1.1,71.9,22.8,1.2,24.0
3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,3.0,1.0,1.0,0.5,0.5,0.5,1.0,...,0.5,0.500000,0.5,FW58,15.7,1.1,71.9,22.8,1.2,24.0
4,0212793B,AimBig Employment BURLEIGH HEADS,MENH,11.0,7.0,3.0,0.5,3.0,0.5,7.0,...,0.5,1.000000,0.5,N687,15.7,1.1,71.9,22.8,1.2,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0212903C,Aimbig Employment RICHMOND,AALL,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.500000,0.5,FX68,17.2,1.3,68.4,21.5,1.1,22.4
204,0212903C,Aimbig Employment WINDSOR,AALL,1.0,0.5,0.5,0.5,0.5,0.5,1.0,...,0.5,0.500000,0.5,FX78,17.2,1.3,68.4,21.5,1.1,22.4
205,0212904D,AimBig Employment HURSTVILLE,AALL,19.0,10.0,3.0,0.5,2.0,0.5,4.0,...,0.5,0.500000,0.5,A981,17.2,1.3,68.4,21.5,1.1,22.4
206,0212904D,AimBig Employment MIRANDA,AALL,9.0,5.0,2.0,0.5,2.0,0.5,4.0,...,0.5,0.333333,0.5,A991,17.2,1.3,68.4,21.5,1.1,22.4


In [67]:
# subsetting the post-quarter actual rates for each contract
df_actPer = df_group.reset_index() # converting groupBy obj -> dataframe for subsetting
df_actPer.columns
df_actPer = df_actPer[['Contract_ID',  'Site_Name',  'specialist_site_type_code', 'actual_13_full',
       'actual_26_full', 'actual_26_path', 'actual_26_wrkast',
       'actual_52_full', 'actual_52_path']]

# subsetting the expected performance for each contract
df_exp_per = df_exp_per.reset_index()
df_exp_per.columns
df_exp_per = df_exp_per[['Contract_ID','Site_Name', 'specialist_site_type_code', 'exPer_13_full', 'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast',
       'exPer_52_full', 'exPer_52_path', '52_full_avg', '52_path_avg',  '26_wrkast_avg', '26_full_avg', '26_path_avg', '13_full_avg']]

# joining the two dataframes
df_perMea = df_actPer.merge(df_exp_per, on=['Contract_ID',  'Site_Name',  'specialist_site_type_code'], how='inner')

df_perMea.columns

df_perMea.isna().sum()
df_perMea.replace([np.inf, -np.inf], 0, inplace=True)

In [68]:
df_perMea

,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,...,exPer_26_path,exPer_26_wrkast,exPer_52_full,exPer_52_path,52_full_avg,52_path_avg,26_wrkast_avg,26_full_avg,26_path_avg,13_full_avg
0,0212789F,AimBig Employment PROSPECT,MENH,66.666667,61.538462,0.0,0.5,33.333333,0.5,3.000000,...,5.0,0.5,0.900000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
1,0212790K,AimBig Employment PROSPECT,MUSK,75.000000,100.000000,0.5,0.5,100.000000,0.5,4.000000,...,0.5,0.5,0.500000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
2,0212791K,AimBig Employment MILE END,MENH,61.538462,66.666667,25.0,0.5,25.000000,0.5,2.166667,...,4.0,0.5,0.500000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,66.666667,100.000000,100.0,0.5,0.500000,0.5,3.000000,...,1.0,0.5,0.500000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
4,0212793B,AimBig Employment BURLEIGH HEADS,MENH,63.636364,57.142857,0.0,0.5,0.000000,0.5,1.571429,...,3.0,0.5,1.000000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0212903C,Aimbig Employment RICHMOND,AALL,0.500000,0.500000,0.5,0.5,0.500000,0.5,0.500000,...,0.5,0.5,0.500000,0.5,17.2,1.3,68.4,21.5,1.1,22.4
204,0212903C,Aimbig Employment WINDSOR,AALL,100.000000,0.500000,0.5,0.5,0.500000,0.5,1.000000,...,0.5,0.5,0.500000,0.5,17.2,1.3,68.4,21.5,1.1,22.4
205,0212904D,AimBig Employment HURSTVILLE,AALL,68.421053,70.000000,0.0,0.5,0.000000,0.5,4.750000,...,0.5,0.5,0.500000,0.5,17.2,1.3,68.4,21.5,1.1,22.4
206,0212904D,AimBig Employment MIRANDA,AALL,66.666667,60.000000,0.0,0.5,0.000000,0.5,2.250000,...,0.5,0.5,0.333333,0.5,17.2,1.3,68.4,21.5,1.1,22.4


### Creating the performance measure scores
 replace weightings - DMS, ESS specific 

In [69]:
df_perMea = df_perMea.fillna(0)

# replace inf with 0 

df_perMea['13_full'] = round(df_perMea['actual_13_full'] * 0.25/df_perMea['exPer_13_full'] * 0.25)
df_perMea['26_full'] = round(df_perMea['actual_26_full'] * 0.35/df_perMea['exPer_26_full'] * 0.35)
df_perMea['26_path'] = round(df_perMea['actual_26_path'] * 0.05/df_perMea['exPer_26_path'] * 0.05)
df_perMea['26_wrkast'] = round(df_perMea['actual_26_wrkast'] * 0.05/df_perMea['exPer_26_wrkast'] * 0.05)
df_perMea['26_comb'] = round((df_perMea['26_full']) + (df_perMea['26_path']) + (df_perMea['26_wrkast']), ndigits=2) # combined ratio - using DMS weightings 

df_perMea['52_full'] = round(df_perMea['actual_52_full'] * 0.25/df_perMea['exPer_52_full'] * 0.25)
df_perMea['52_path'] = round(df_perMea['actual_52_path'] * 0.05/df_perMea['exPer_52_path'] * 0.05)
df_perMea['52_comb'] = round((df_perMea['52_full']) + (df_perMea['52_path'])) # combined ratio - using DMS weightings 

# for the organisational averages (Sub368)
df_perMea['13_full_avg'] = round(df_perMea['actual_13_full'] * 0.25/df_perMea['13_full_avg'] * 0.25)
df_perMea['26_full_avg'] = round(df_perMea['actual_26_full'] * 0.35/df_perMea['26_full_avg'] * 0.35)
df_perMea['26_path_avg'] = round(df_perMea['actual_26_path'] * 0.05/df_perMea['26_path_avg'] * 0.05)
df_perMea['26_wrkast_avg'] = round(df_perMea['actual_26_wrkast'] * 0.05/df_perMea['26_wrkast_avg'] * 0.05)
df_perMea['26_comb_avg'] = round((df_perMea['26_full_avg']) + (df_perMea['26_path_avg']) + (df_perMea['26_wrkast_avg']), ndigits=2) # combined ratio - using DMS weightings 

df_perMea['52_full_avg'] = round(df_perMea['actual_52_full'] * 0.25/df_perMea['52_full_avg'] * 0.25)
df_perMea['52_path_avg'] = round(df_perMea['actual_52_path'] * 0.05/df_perMea['52_path_avg'] * 0.05)
df_perMea['52_comb_avg'] = round((df_perMea['52_full_avg'] * 0.25) + (df_perMea['52_path_avg'])) # combined ratio - using DMS weightings 

# replace infinity -> 

df_perMea.columns

Index(['Contract_ID', 'Site_Name', 'specialist_site_type_code',
       'actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path', '52_full_avg', '52_path_avg', '26_wrkast_avg',
       '26_full_avg', '26_path_avg', '13_full_avg', '13_full', '26_full',
       '26_path', '26_wrkast', '26_comb', '52_full', '52_path', '52_comb',
       '26_comb_avg', '52_comb_avg'],
      dtype='object')

In [70]:
df_perMea

,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,...,13_full,26_full,26_path,26_wrkast,26_comb,52_full,52_path,52_comb,26_comb_avg,52_comb_avg
0,0212789F,AimBig Employment PROSPECT,MENH,66.666667,61.538462,0.0,0.5,33.333333,0.5,3.000000,...,1.0,4.0,0.0,0.0,4.0,2.0,0.0,2.0,0.0,0.0
1,0212790K,AimBig Employment PROSPECT,MUSK,75.000000,100.000000,0.5,0.5,100.000000,0.5,4.000000,...,1.0,12.0,0.0,0.0,12.0,12.0,0.0,12.0,1.0,0.0
2,0212791K,AimBig Employment MILE END,MENH,61.538462,66.666667,25.0,0.5,25.000000,0.5,2.166667,...,2.0,7.0,0.0,0.0,7.0,3.0,0.0,3.0,0.0,0.0
3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,66.666667,100.000000,100.0,0.5,0.500000,0.5,3.000000,...,1.0,12.0,0.0,0.0,12.0,0.0,0.0,0.0,1.0,0.0
4,0212793B,AimBig Employment BURLEIGH HEADS,MENH,63.636364,57.142857,0.0,0.5,0.000000,0.5,1.571429,...,3.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0212903C,Aimbig Employment RICHMOND,AALL,0.500000,0.500000,0.5,0.5,0.500000,0.5,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204,0212903C,Aimbig Employment WINDSOR,AALL,100.000000,0.500000,0.5,0.5,0.500000,0.5,1.000000,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,0212904D,AimBig Employment HURSTVILLE,AALL,68.421053,70.000000,0.0,0.5,0.000000,0.5,4.750000,...,1.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
206,0212904D,AimBig Employment MIRANDA,AALL,66.666667,60.000000,0.0,0.5,0.000000,0.5,2.250000,...,2.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0


### Step 4: Standardise the performance measure scores
Ratio of actual rate pre-standardisation and post-standardisation.

    - scale 0 to 4
    - Using MinMaxScalar
    - Only the combined outcomes - 13, 26, 52

In [71]:
df_perMea


,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,...,13_full,26_full,26_path,26_wrkast,26_comb,52_full,52_path,52_comb,26_comb_avg,52_comb_avg
0,0212789F,AimBig Employment PROSPECT,MENH,66.666667,61.538462,0.0,0.5,33.333333,0.5,3.000000,...,1.0,4.0,0.0,0.0,4.0,2.0,0.0,2.0,0.0,0.0
1,0212790K,AimBig Employment PROSPECT,MUSK,75.000000,100.000000,0.5,0.5,100.000000,0.5,4.000000,...,1.0,12.0,0.0,0.0,12.0,12.0,0.0,12.0,1.0,0.0
2,0212791K,AimBig Employment MILE END,MENH,61.538462,66.666667,25.0,0.5,25.000000,0.5,2.166667,...,2.0,7.0,0.0,0.0,7.0,3.0,0.0,3.0,0.0,0.0
3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,66.666667,100.000000,100.0,0.5,0.500000,0.5,3.000000,...,1.0,12.0,0.0,0.0,12.0,0.0,0.0,0.0,1.0,0.0
4,0212793B,AimBig Employment BURLEIGH HEADS,MENH,63.636364,57.142857,0.0,0.5,0.000000,0.5,1.571429,...,3.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0212903C,Aimbig Employment RICHMOND,AALL,0.500000,0.500000,0.5,0.5,0.500000,0.5,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204,0212903C,Aimbig Employment WINDSOR,AALL,100.000000,0.500000,0.5,0.5,0.500000,0.5,1.000000,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,0212904D,AimBig Employment HURSTVILLE,AALL,68.421053,70.000000,0.0,0.5,0.000000,0.5,4.750000,...,1.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
206,0212904D,AimBig Employment MIRANDA,AALL,66.666667,60.000000,0.0,0.5,0.000000,0.5,2.250000,...,2.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0


In [72]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,4))
# subset the dataframe, then rejoin -> RESET_INDEX first to transform into dataframe, then rejoin, then groupBy

included_columns = ['13_full', '26_comb', '52_comb', '13_full_avg', '26_comb_avg', '52_comb_avg']


columns_to_scale = df_perMea.columns[df_perMea.columns.isin(included_columns)]

scaledCols = scaler.fit_transform(df_perMea.loc[:, columns_to_scale])

df_perMea.loc[:, columns_to_scale] = scaledCols
df_perMea


,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,...,13_full,26_full,26_path,26_wrkast,26_comb,52_full,52_path,52_comb,26_comb_avg,52_comb_avg
0,0212789F,AimBig Employment PROSPECT,MENH,66.666667,61.538462,0.0,0.5,33.333333,0.5,3.000000,...,0.333333,4.0,0.0,0.0,0.516129,2.0,0.0,0.258065,0.0,0.0
1,0212790K,AimBig Employment PROSPECT,MUSK,75.000000,100.000000,0.5,0.5,100.000000,0.5,4.000000,...,0.333333,12.0,0.0,0.0,1.548387,12.0,0.0,1.548387,4.0,0.0
2,0212791K,AimBig Employment MILE END,MENH,61.538462,66.666667,25.0,0.5,25.000000,0.5,2.166667,...,0.666667,7.0,0.0,0.0,0.903226,3.0,0.0,0.387097,0.0,0.0
3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,66.666667,100.000000,100.0,0.5,0.500000,0.5,3.000000,...,0.333333,12.0,0.0,0.0,1.548387,0.0,0.0,0.000000,4.0,0.0
4,0212793B,AimBig Employment BURLEIGH HEADS,MENH,63.636364,57.142857,0.0,0.5,0.000000,0.5,1.571429,...,1.000000,4.0,0.0,0.0,0.516129,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0212903C,Aimbig Employment RICHMOND,AALL,0.500000,0.500000,0.5,0.5,0.500000,0.5,0.500000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
204,0212903C,Aimbig Employment WINDSOR,AALL,100.000000,0.500000,0.5,0.5,0.500000,0.5,1.000000,...,2.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
205,0212904D,AimBig Employment HURSTVILLE,AALL,68.421053,70.000000,0.0,0.5,0.000000,0.5,4.750000,...,0.333333,3.0,0.0,0.0,0.387097,0.0,0.0,0.000000,0.0,0.0
206,0212904D,AimBig Employment MIRANDA,AALL,66.666667,60.000000,0.0,0.5,0.000000,0.5,2.250000,...,0.666667,9.0,0.0,0.0,1.161290,0.0,0.0,0.000000,0.0,0.0


### Step 5: Calculate the Overall Performance Score
- Multiplying the standardised performance scores with the DMS/ESS weighting
TO-DO: multiply with the respective DMS/ESS weight

In [73]:
df_perMea['13_full'] = df_perMea['13_full'] * 0.228
df_perMea['26_comb'] = df_perMea['26_comb'] * 0.457
df_perMea['52_comb'] = df_perMea['52_comb'] * 0.285

df_perMea['overall_perScore'] = df_perMea['13_full'] + df_perMea['26_comb'] + df_perMea['52_comb']


#avg performance score -> against national average
df_perMea['13_full_avg'] = df_perMea['13_full_avg'] * 0.228
df_perMea['26_comb_avg'] = df_perMea['26_comb_avg'] * 0.457
df_perMea['52_comb_avg'] = df_perMea['52_comb_avg'] * 0.285

df_perMea['overall_perScore_avg'] = df_perMea['13_full_avg'] + df_perMea['26_comb_avg'] + df_perMea['52_comb_avg']



### Step 6: Calculate the Star Rating Percentage
- Calculated using the formula:
str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 


str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 

In [74]:
df_perMea = df_perMea.reset_index()
df_perMea.columns

Index(['index', 'Contract_ID', 'Site_Name', 'specialist_site_type_code',
       'actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path', '52_full_avg', '52_path_avg', '26_wrkast_avg',
       '26_full_avg', '26_path_avg', '13_full_avg', '13_full', '26_full',
       '26_path', '26_wrkast', '26_comb', '52_full', '52_path', '52_comb',
       '26_comb_avg', '52_comb_avg', 'overall_perScore',
       'overall_perScore_avg'],
      dtype='object')

In [75]:
df_perMea

,index,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,...,26_path,26_wrkast,26_comb,52_full,52_path,52_comb,26_comb_avg,52_comb_avg,overall_perScore,overall_perScore_avg
0,0,0212789F,AimBig Employment PROSPECT,MENH,66.666667,61.538462,0.0,0.5,33.333333,0.5,...,0.0,0.0,0.235871,2.0,0.0,0.073548,0.000,0.0,0.385419,0.000
1,1,0212790K,AimBig Employment PROSPECT,MUSK,75.000000,100.000000,0.5,0.5,100.000000,0.5,...,0.0,0.0,0.707613,12.0,0.0,0.441290,1.828,0.0,1.224903,1.828
2,2,0212791K,AimBig Employment MILE END,MENH,61.538462,66.666667,25.0,0.5,25.000000,0.5,...,0.0,0.0,0.412774,3.0,0.0,0.110323,0.000,0.0,0.675097,0.000
3,3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,66.666667,100.000000,100.0,0.5,0.500000,0.5,...,0.0,0.0,0.707613,0.0,0.0,0.000000,1.828,0.0,0.783613,1.828
4,4,0212793B,AimBig Employment BURLEIGH HEADS,MENH,63.636364,57.142857,0.0,0.5,0.000000,0.5,...,0.0,0.0,0.235871,0.0,0.0,0.000000,0.000,0.0,0.463871,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,203,0212903C,Aimbig Employment RICHMOND,AALL,0.500000,0.500000,0.5,0.5,0.500000,0.5,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000,0.000
204,204,0212903C,Aimbig Employment WINDSOR,AALL,100.000000,0.500000,0.5,0.5,0.500000,0.5,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.456000,0.000
205,205,0212904D,AimBig Employment HURSTVILLE,AALL,68.421053,70.000000,0.0,0.5,0.000000,0.5,...,0.0,0.0,0.176903,0.0,0.0,0.000000,0.000,0.0,0.252903,0.000
206,206,0212904D,AimBig Employment MIRANDA,AALL,66.666667,60.000000,0.0,0.5,0.000000,0.5,...,0.0,0.0,0.530710,0.0,0.0,0.000000,0.000,0.0,0.682710,0.000


In [76]:
# calculating the star rating percentage
df_perMea['star_rating_perc'] = (df_perMea['overall_perScore'] - df_perMea['overall_perScore_avg'])/ (df_perMea['overall_perScore_avg']) * 100

In [77]:
df_perMea.to_csv('C:/Users/AManalo/star_ratings_new/star_ratings.csv')

PermissionError: [Errno 13] Permission denied: 'C:/Users/AManalo/star_ratings_new/star_ratings.csv'

In [ ]:
df_perMea

,index,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,...,26_wrkast,26_comb,52_full,52_path,52_comb,26_comb_avg,52_comb_avg,overall_perScore,overall_perScore_avg,star_rating_perc
0,0,0212789F,AimBig Employment PROSPECT,MENH,66.666667,61.538462,0.0,0.5,33.333333,0.5,...,0.0,0.206452,2.0,0.0,0.064516,0.0,0.0,0.337634,0.0,inf
1,1,0212790K,AimBig Employment PROSPECT,MUSK,75.000000,100.000000,0.5,0.5,100.000000,0.5,...,0.0,0.619355,12.0,0.0,0.387097,1.6,0.0,1.073118,1.6,-32.930108
2,2,0212791K,AimBig Employment MILE END,MENH,61.538462,66.666667,25.0,0.5,25.000000,0.5,...,0.0,0.361290,3.0,0.0,0.096774,0.0,0.0,0.591398,0.0,inf
3,3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,66.666667,100.000000,100.0,0.5,0.500000,0.5,...,0.0,0.619355,0.0,0.0,0.000000,1.6,0.0,0.686022,1.6,-57.123656
4,4,0212793B,AimBig Employment BURLEIGH HEADS,MENH,63.636364,57.142857,0.0,0.5,0.000000,0.5,...,0.0,0.206452,0.0,0.0,0.000000,0.0,0.0,0.406452,0.0,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,203,0212903C,Aimbig Employment RICHMOND,AALL,0.500000,0.500000,0.5,0.5,0.500000,0.5,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,NaN
204,204,0212903C,Aimbig Employment WINDSOR,AALL,100.000000,0.500000,0.5,0.5,0.500000,0.5,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.400000,0.0,inf
205,205,0212904D,AimBig Employment HURSTVILLE,AALL,68.421053,70.000000,0.0,0.5,0.000000,0.5,...,0.0,0.154839,0.0,0.0,0.000000,0.0,0.0,0.221505,0.0,inf
206,206,0212904D,AimBig Employment MIRANDA,AALL,66.666667,60.000000,0.0,0.5,0.000000,0.5,...,0.0,0.464516,0.0,0.0,0.000000,0.0,0.0,0.597849,0.0,inf
